In [ ]:
!pip install opencv-python-headless

In [ ]:
# conectar a google drive
from google.colab import drive
drive.mount('/content/drive')

## Suma de imágenes

Logos: https://www.kaggle.com/datasets/kkhandekar/popular-brand-logos-image-dataset

1481 popular brand logos

UCF-QNRF_ECCV18: https://www.kaggle.com/datasets/kokunsyu/ucf-qnrf-eccv18

 Dataset utilizado para la tarea de conteo de multitudes (crowd counting). Contiene imágenes de escenas densamente pobladas, como manifestaciones y eventos públicos.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar las imágenes
imagen_principal = cv2.imread('/content/drive/MyDrive/datos_PDI/logos/img_0002.jpg')
logo = cv2.imread('/content/drive/MyDrive/datos_PDI/logos/Heineken-logo-400x400.png')

# Convertir las imágenes de BGR a RGB (OpenCV usa BGR por defecto)
imagen_principal_rgb = cv2.cvtColor(imagen_principal, cv2.COLOR_BGR2RGB)
logo_rgb = cv2.cvtColor(logo, cv2.COLOR_BGR2RGB)

# Obtener el tamaño de la imagen principal
alto, ancho = imagen_principal_rgb.shape[:2]

# Redimensionar el logo al tamaño de la imagen principal
logo_redimensionado = cv2.resize(logo_rgb, (ancho, alto))

# Definir pesos para la combinación ponderada
peso_imagen_principal = 0.7
peso_logo = 1 - peso_imagen_principal

# Combinar las imágenes usando una suma ponderada
imagen_combinada = cv2.addWeighted(imagen_principal_rgb, peso_imagen_principal, logo_redimensionado, peso_logo, 0)

# Mostrar las imágenes usando matplotlib
plt.figure(figsize=(16, 12))

plt.subplot(1, 3, 1)
plt.title('Imagen Principal')
plt.imshow(imagen_principal_rgb)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Logo Redimensionado')
plt.imshow(logo_redimensionado)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Imagen Combinada')
plt.imshow(imagen_combinada)
plt.axis('off')

plt.show()

## Diferencia de imágenes

Ejemplo de substracción de fondo para detección de movimiento e intrusos

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Ruta del video
video_path = '/content/drive/MyDrive/datos_PDI/ladron.mp4'

# Abrir el video
cap = cv2.VideoCapture(video_path)

# Leer los primeros 60 frames para obtener el fondo inicial
for _ in range(60):
    ret, fondo = cap.read()
if not ret:
    print("Error al leer el video.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Convertir el último frame de los primeros 60 frames a escala de grises
fondo = cv2.cvtColor(fondo, cv2.COLOR_BGR2GRAY)

# Número de frames a procesar
num_frames_to_process = 10

# Crear una figura con 2 filas y 5 columnas
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Convertir el objeto de ejes a una lista para iterar fácilmente
axes = axes.flatten()

for i in range(num_frames_to_process):
    # Dejar pasar 50 frames
    for _ in range(50):
        ret, frame = cap.read()
        if not ret:
            break

    # Convertir el frame actual a escala de grises
    gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calcular la diferencia entre el fondo y el frame actual
    diferencia = cv2.absdiff(fondo, gris)

    # Aplicar un umbral para obtener una imagen binaria
    _, umbral = cv2.threshold(diferencia, 50, 255, cv2.THRESH_BINARY)


    # Encontrar contornos en la imagen binaria
    contornos, _ = cv2.findContours(umbral, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Inicializar el bounding box más grande
    bbox_max = None
    max_area = 0
    # Iterar sobre todos los contornos para encontrar el bounding box más grande
    for contorno in contornos:
        (x, y, w, h) = cv2.boundingRect(contorno)
        area = w * h
        if area > max_area:
            max_area = area
            bbox_max = (x, y, w, h)
    # Dibujar el bounding box más grande
    if bbox_max is not None:
        (x, y, w, h) = bbox_max
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
    # Convertir el frame de BGR a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


    # Mostrar la imagen diferencia entre fondo y actual, en escala de grises
    axes[i].imshow(diferencia, cmap='gray')
    # Mostrar el frame en la cuadrícula
    #axes[i].imshow(frame_rgb)

    axes[i].axis('off')

# Ajustar el layout para evitar solapamiento
plt.tight_layout()

# Mostrar la figura con todos los frames
plt.show()

# Liberar recursos
cap.release()
cv2.destroyAllWindows()

## Segmentación color y multiplicación

**Corn or Maize Leaf Disease Dataset**

A dataset for classification of corn or maize plant leaf diseases

Dataset Description:
- 0: Common Rust - 1306 images
- 1: Gray Leaf Spot - 574 images
- 2: Blight -1146 images
- 3: Healthy - 1162 images

https://www.kaggle.com/datasets/smaranjitghose/corn-or-maize-leaf-disease-dataset

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from zipfile import ZipFile
import os

# Ruta a una imagen de ejemplo
image_path = '/content/drive/MyDrive/datos_PDI/Corn_Maize_Leaf/Corn_Health (1023).jpg'

# Cargar la imagen
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convertir a espacio de color HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Definir los rangos de color para segmentar el color verde
# LOCALIZACION EN HSV:
# H: 120º en la rueda, pero en OpenCV va de 0-180º --> el G está centrado en 60º
# S: desde un rango bajo a máximo (para cubrirnos)
# V: ídem S
lower_green = np.array([25, 40, 40])
upper_green = np.array([80, 255, 255])

# Crear una máscara con los colores definidos
mask = cv2.inRange(hsv, lower_green, upper_green)

# Aplicar la máscara a la imagen original
segmented_image = cv2.bitwise_and(image_rgb, image_rgb, mask=mask)

# Mostrar la imagen original y la segmentada
plt.figure(figsize=(6, 4))

plt.subplot(1, 2, 1)
plt.title('Imagen Original')
plt.imshow(image_rgb)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Imagen Segmentada')
plt.imshow(segmented_image)
plt.axis('off')

plt.show()

**PREGUNTAS**

- ¿Por qué se producen los huecos en la imagen segmentada?
- ¿Qué (buen) comportamiento observa en la segmentación de la imagen saludable 1019?

   TIP: está relacionada a la ventaja del desacoplamiento "cromaticidad / brillo" que hace el modelo HSV respecto al RGB.
- ¿Cómo podría utilizar la segmentación color para hacer un "clasificador de salud" de las hojas?

## PROYECTO

**Clasificación de frutas**

Dataset: "Fruits fresh and rotten for classification"

(parte del dataset "Fresh and Rotten Classification")

https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Definir las rutas a las carpetas de train y test
train_dir = '/content/drive/MyDrive/datos_PDI/Fruits_classification/train'
test_dir = '/content/drive/MyDrive/datos_PDI/Fruits_classification/test'
classes = ['roja', 'verde']
image_size = (100, 100)  # Tamaño al que redimensionaremos las imágenes


def cambiar_cercanos_al_blanco_a_negro(img, umbral=240):
    """
    Cambia los valores cercanos al blanco por negro en la imagen.

    :param img: Imagen en formato BGR.
    :param umbral: Valor del canal de intensidad (V) en HSV. Los píxeles con valores
                   de intensidad mayores que este umbral se convertirán en negro.
    :return: Imagen con los valores cercanos al blanco cambiados a negro.
    """
    # Convertir la imagen de BGR a HSV
    hsv_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Crear una máscara donde los valores de intensidad (V) son mayores que el umbral
    _, _, V = cv2.split(hsv_image)
    mascara = V > umbral
    # Crear una imagen de salida donde los valores cercanos al blanco se cambian a negro
    imagen_salida = img.copy()
    imagen_salida[mascara] = [0, 0, 0]
    return imagen_salida

# Función para cargar y redimensionar imágenes
def load_and_resize_images(base_dir, classes, num_images, size):
    images = []
    labels = []

    for fruit in classes:
        for i in range(1, num_images + 1):
            img_path = os.path.join(base_dir, f'{fruit}{i}.png')
            img = cv2.imread(img_path)
            # Verificar que la imagen se ha cargado correctamente
            if img is not None:
                img = cambiar_cercanos_al_blanco_a_negro(img)
                # Redimensionar imagen
                img_resized = cv2.resize(img, size)
                images.append(img_resized)
                labels.append(fruit)
            else:
                print(f"ERROR: no se pudo cargar imagen {img_path}")
    return images, labels

# Cargar y redimensionar imágenes y etiquetas de entrenamiento (5 imágenes por fruta)
train_images, train_labels = load_and_resize_images(train_dir, classes, num_images=5, size=image_size)

# Cargar y redimensionar imágenes y etiquetas de prueba (2 imágenes por fruta)
test_images, test_labels = load_and_resize_images(test_dir, classes, num_images=2, size=image_size)

# Configurar la visualización
num_classes = len(classes)
num_images_per_class = len([label for label in train_labels if label == classes[0]])

plt.figure(figsize=(15, num_classes * 3))  # Ajustar el tamaño según sea necesario

for idx, fruit in enumerate(classes):
    # Encontrar los índices de las imágenes de la fruta actual
    fruit_indices = [i for i, label in enumerate(train_labels) if label == fruit]
    # Mostrar todas las imágenes de la fruta en la fila correspondiente
    for j, image_idx in enumerate(fruit_indices):
        plt.subplot(num_classes, num_images_per_class, idx * num_images_per_class + j + 1)
        plt.imshow(cv2.cvtColor(train_images[image_idx], cv2.COLOR_BGR2RGB))
        plt.axis('off')
        if j == 0:
            plt.title(fruit.capitalize())

plt.show()

In [ ]:
len(train_images)

In [ ]:
print(f"train_images shape: {np.array(train_images).shape}")

In [ ]:
train_labels

In [ ]:
# mostrar imagen 1 de train_images, con su label

plt.imshow(cv2.cvtColor(train_images[0], cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

train_labels[0]

**PROGRAMAR AQUÍ LA SOLUCIÓN**:

In [ ]:
# Solución propuesta

....

**Análisis de resultados**

Explicar:
- Extracción de características: cuál fue la aproximación, pasos seguidos (diagrama en bloque) y parámetros elegidos.
- Clasificación: mencionar la técnica utilizada -no explicar teóricamente-.
- Evaluación: explicar cómo se llevó a cabo el análisis del rendimiento del modelo (proceso elegido y resultados).
- Mostrar algunos ejemplos de predicciones correctas y algunos de incorrectas (si surgieran).
